In [ ]:
from tqdm import tqdm
import json

from mmagent.videograph import VideoGraph
from mmagent.utils.general import *
from mmagent.utils.video_processing import *
from mmagent.utils.chat_api import *
from mmagent.prompts import *

from mmagent.retrieve import answer_with_retrieval
from data_preparation.generate_memory import streaming_process_video

processing_config = json.load(open("configs/processing_config.json"))
memory_config = json.load(open("configs/memory_config.json"))

In [ ]:
# video paths can be paths to directories or paths to mp4 files
video_paths = ["path/to/clips/files"]
save_dir = processing_config["save_dir"]
max_workers = 1


def process_single_video(video_path):
    video_graph = VideoGraph(**memory_config)
    streaming_process_video(video_graph, video_path)


# Process videos in parallel using ThreadPoolExecutor with max_workers limit
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    # Process videos in parallel using map
    list(
        tqdm(
            executor.map(process_single_video, video_paths),
            total=len(video_paths),
            desc="Processing videos",
        )
    )

In [ ]:
video_graph_path = (
    "path/to/mem/pkls"
)
video_graph = load_video_graph(video_graph_path)

question = "How might the social development of children living in military families be affected?"

answer = answer_with_retrieval(
    video_graph,
    question,
    query_num=processing_config["query_num"],
    topk=processing_config["topk"],
    mode="argmax",
)

In [ ]:
video_graph_path = "path/to/mem/pkls"
video_graph = load_video_graph(video_graph_path)
video_graph.visualize()

# print all episodic and semantic nodes
for node_id in video_graph.text_nodes:
    print(video_graph.nodes[node_id].metadata["contents"][0])

In [ ]:
import numpy as np
from tqdm import tqdm
import os
import json
from concurrent.futures import ProcessPoolExecutor
import multiprocessing

from mmagent.videograph import VideoGraph
from mmagent.utils.general import *
from mmagent.utils.video_processing import *
from mmagent.utils.chat_api import *
from mmagent.prompts import *

from mmagent.face_processing import process_faces
from mmagent.voice_processing import process_voices
from mmagent.memory_processing import (
    process_captions,
    generate_captions_and_thinkings_with_ids,
)
from data_preparation.generate_memory import streaming_process_video

processing_config = json.load(open("configs/processing_config.json"))
memory_config = json.load(open("configs/memory_config.json"))

video_paths = [
    "path/to/clips/files"
]

save_dir = processing_config["save_dir"]
generated_memories = os.listdir(save_dir)
generated_memories = [
    generated_memory
    for generated_memory in generated_memories
    if generated_memory.endswith(".pkl")
]
video_paths = [
    video_path
    for video_path in video_paths
    if generate_file_name(video_path) + ".pkl" not in generated_memories
]

cpu_count = multiprocessing.cpu_count()
max_workers = min(cpu_count, processing_config.get("max_parallel_videos", 4))

print(f"Using {max_workers} processes (CPU cores: {cpu_count})")

preprocessing = []


def process_single_video(video_path):
    video_graph = VideoGraph(**memory_config)
    try:
        streaming_process_video(video_graph, video_path, preprocessing=preprocessing)
    except Exception as e:
        log_dir = processing_config["log_dir"]
        os.makedirs(log_dir, exist_ok=True)
        with open(os.path.join(log_dir, f"generate_memory_error.log"), "a") as f:
            f.write(f"Error processing video {video_path}: {e}\n")


with ProcessPoolExecutor(max_workers=max_workers) as executor:
    list(
        tqdm(
            executor.map(process_single_video, video_paths),
            total=len(video_paths),
            desc="Processing videos",
        )
    )

In [11]:
from mmagent.utils.general import load_video_graph
from mmagent.retrieve import answer_with_retrieval
import sys
import mmagent.videograph

sys.modules["videograph"] = mmagent.videograph

video_graph_path = "path/to/mem/pkls"
video_graph = load_video_graph(video_graph_path)
# video_graph.visualize()

video_graph.refresh_equivalences()
question = "Do Hannahlei's parents support her dancing?"
answer, session = answer_with_retrieval(video_graph, question)
print(session[0])

100%|██████████| 1467/1467 [00:00<00:00, 19829.53it/s]


[[], [{'clip_id': 'CLIP_14', 'memory': ['<character_1> speaks into a microphone on a stage with blue background and white vertical bars of light.', '<character_1> stands next to <character_1>, smiling.', '<character_1> says that SMS voting is available on all wireless carriers, and message and data rates may apply.', '<character_1> directs viewers to fox.com/dance for voting details, but asks them not to vote until the end of the show.', '<character_1> says that the judges have given their feedback, but some other people have something to say to <character_1>.', '<character_1> tells <character_1> to take a look.', 'A video of <character_1> at the beach is shown.', '<character_1>, wearing a light blue two-piece outfit, dances on the beach.', "<character_1>'s parents appear in a video, sitting on a couch in front of a piano.", "<character_1>'s mother says that <character_1> was a very shy child.", "<character_1>'s parents are supportive of her dance career.", '<character_2> encourages vi